ref: https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

Ref: https://women.trueid.net/detail/kEN97vDvokbn?utm_source=web-trueid&utm_medium=ctw&utm_term=clicklink&utm_campaign=women_4RQeYK5bOmMR_relatecontent_women_kEN97vDvokbn_15/02/2021

## Install Dependency

In [1]:
!pip install pythainlp==3.0.5

     |████████████████████████████████| 11.5 MB 5.1 MB/s 


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras
import tensorflow as tf
tf.random.set_seed(1234)
from numpy.random import seed
# set_random_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")

from pythainlp import word_tokenize
pd.set_option("max_colwidth", -1)

In [3]:
df = pd.read_csv("siaw_caption.txt", names=["text"])

In [4]:
df.head()

,text
0,รักของเราอาจไม่หวาน แต่คุณไม่ต้องหารกับใครนะ
1,ถ้าเธอยังไม่เข้านอน ลองเข้ามาในใจเราก่อนได้นะ
2,อยากให้เธอกินคลีนจัง เธอจะได้ลดของมัน แล้วมาเป็นของเรา
3,หลงตัวเองนิดหน่อย แต่ที่หลงบ่อยๆ ก็เธอนั่นแหละ
4,ชอบตื่นเช้าอ่ะ เพราะไม่อยากรักเธอในวันที่สายเกินไป


## Tokenize thai sentence

In [5]:
df["tokens"] = df["text"].apply(lambda text: word_tokenize(text, keep_whitespace=True))

In [19]:
df["processed_tokens"] = df["tokens"].apply(lambda tokens: tokens + ["END"])
df["processed_sentence"] = df["processed_tokens"].apply(lambda tokens: " ".join(tokens))

In [20]:
df[["processed_sentence"]]

,processed_sentence
0,รัก ของ เรา อาจ ไม่ หวาน แต่ คุณ ไม่ต้อง หาร กับ ใคร นะ END
1,ถ้า เธอ ยัง ไม่ เข้านอน ลอง เข้ามา ใน ใจ เรา ก่อน ได้ นะ END
2,อยาก ให้ เธอ กิน คลี น จัง เธอ จะ ได้ ลด ของ มัน แล้ว มา เป็น ของ เรา END
3,หลงตัวเอง นิดหน่อย แต่ ที่ หลง บ่อยๆ ก็ เธอ นั่นแหละ END
4,ชอบ ตื่น เช้า อ่ะ เพราะ ไม่ อยาก รัก เธอ ใน วันที่ สาย เกินไป END
...,...
295,ใคร มัน จะ อยู่ ข้าง เรา ได้ดี เท่า เงา ของ ตัวเรา เอง END
296,โสด สวย สต รอง ถ้า เธอ ได้ มอง แล้ วจะ เสียดาย END
297,โสด แล้ว ยิ้ม ดีกว่า ถูก ทิ้ง แล้ว ร้องไห้ END
298,เป็นโสด อย่าง อิสระ ดีกว่า เข้าไป เป็น ภาระ หัวใจ ใคร END


In [21]:
tokenizer = Tokenizer(lower=False)

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(df["processed_sentence"])

In [22]:
tokenizer.sequences_to_texts(inp_sequences[:10])

['รัก ของ',
 'รัก ของ เรา',
 'รัก ของ เรา อาจ',
 'รัก ของ เรา อาจ ไม่',
 'รัก ของ เรา อาจ ไม่ หวาน',
 'รัก ของ เรา อาจ ไม่ หวาน แต่',
 'รัก ของ เรา อาจ ไม่ หวาน แต่ คุณ',
 'รัก ของ เรา อาจ ไม่ หวาน แต่ คุณ ไม่ต้อง',
 'รัก ของ เรา อาจ ไม่ หวาน แต่ คุณ ไม่ต้อง หาร',
 'รัก ของ เรา อาจ ไม่ หวาน แต่ คุณ ไม่ต้อง หาร กับ']

In [23]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors = input_sequences[:,:-1]
    label = input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [24]:
max_sequence_len

22

In [25]:
predictors

array([[  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0,  20,  26],
       [  0,   0,   0, ...,  20,  26,   4],
       ...,
       [  0,   0,   0, ...,   7, 355,  30],
       [  0,   0,   0, ..., 355,  30,  24],
       [  0,   0,   0, ...,  30,  24, 896]], dtype=int32)

In [26]:
total_words

897

In [27]:
label.shape

(3577, 897)

## Define structure and train model.

In [28]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    model.add(Embedding(total_words, output_dim=128, input_length=input_len))
    
    # model.add(LSTM(100))
    # model.add(Dropout(0.1))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 21, 128)           114816    
                                                                 
 lstm_6 (LSTM)               (None, 21, 256)           394240    
                                                                 
 dropout_6 (Dropout)         (None, 21, 256)           0         
                                                                 
 lstm_7 (LSTM)               (None, 21, 256)           525312    
                                                                 
 dropout_7 (Dropout)         (None, 21, 256)           0         
                                                                 
 lstm_8 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_8 (Dropout)         (None, 128)              

In [29]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(predictors, label, epochs=500, verbose=1, callbacks=[early_stopping])

Epoch 1/500
112/112 [==============================] - 10s 32ms/step - loss: 5.8813
Epoch 2/500
112/112 [==============================] - 4s 34ms/step - loss: 5.5805
Epoch 3/500
112/112 [==============================] - 4s 31ms/step - loss: 5.5503
Epoch 4/500
112/112 [==============================] - 4s 32ms/step - loss: 5.5258
Epoch 5/500
112/112 [==============================] - 3s 31ms/step - loss: 5.4728
Epoch 6/500
112/112 [==============================] - 3s 29ms/step - loss: 5.4009
Epoch 7/500
112/112 [==============================] - 3s 29ms/step - loss: 5.3571
Epoch 8/500
112/112 [==============================] - 3s 29ms/step - loss: 5.2856
Epoch 9/500
112/112 [==============================] - 3s 29ms/step - loss: 5.2311
Epoch 10/500
112/112 [==============================] - 3s 29ms/step - loss: 5.1604
Epoch 11/500
112/112 [==============================] - 3s 29ms/step - loss: 5.0870
Epoch 12/500
112/112 [==============================] - 3s 29ms/step - loss: 5.0271


KeyboardInterrupt: ignored

In [ ]:
# def generate_text(seed_text, next_words, model, max_sequence_len):
#     for _ in range(next_words):
#         token_list = tokenizer.texts_to_sequences([seed_text])[0]
#         token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#         predicted = model.predict(token_list, verbose=0)
#         # predicted = predicted.argmax()
#         predicted = np.random.choice(np.arange(0, predicted.shape[1]), p=predicted[0])

#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted:
#                 output_word = word
#                 break
#         seed_text += " "+output_word
#     return seed_text.title()

def generate_text(seed_text, model, max_sequence_len):
    counter = 0
    while True:
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        # predicted = predicted.argmax()
        predicted = np.random.choice(np.arange(0, predicted.shape[1]), p=predicted[0])

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
        if output_word == "end" or counter > 100:
          return seed_text
        counter += 1
    return seed_text.title()

In [ ]:
test_text = "ของหวาน"
test_input = " ".join(word_tokenize(test_text))
print(generate_text(test_input, model, max_sequence_len))

ของหวาน มี เรียว แต่ เจอ ใน ใจ เรา อยาก กิน เธอ end


In [ ]:
token_list = tokenizer.texts_to_sequences([test_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
predicted = model.predict(token_list, verbose=0)

In [ ]:
np.arange(0, predicted.shape[1])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18